<a href="https://colab.research.google.com/github/wel51x/DS-Unit-2-Sprint-2-Linear-Regression/blob/master/Polynomial_Log_linear_Regression_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
# seaborn
!pip3 install seaborn -U

Requirement already up-to-date: seaborn in /usr/local/lib/python3.6/dist-packages (0.9.0)


In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from mpl_toolkits.mplot3d import Axes3D

In [109]:
df = pd.read_csv('https://raw.githubusercontent.com/ryanleeallred/datasets/master/Ames%20Housing%20Data/train.csv')
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
print(df.shape)
df.sample(8)

(1460, 81)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
600,601,60,RL,74.0,10927,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NridgHt,Norm,Norm,1Fam,2Story,8,5,2005,2005,Gable,CompShg,VinylSd,VinylSd,BrkFace,280.0,Gd,TA,PConc,Gd,TA,Av,GLQ,546,Unf,0,512,1058,GasA,Ex,Y,SBrkr,1058,846,0,1904,1,0,2,1,3,1,Ex,8,Typ,1,Gd,BuiltIn,2003.0,Fin,2,736,TA,TA,Y,179,60,0,0,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal,275000
1226,1227,60,RL,86.0,14598,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,Somerst,Feedr,Norm,1Fam,2Story,6,5,2007,2007,Gable,CompShg,VinylSd,VinylSd,Stone,74.0,Gd,TA,PConc,Gd,TA,Mn,Unf,0,Unf,0,894,894,GasA,Ex,Y,SBrkr,894,1039,0,1933,0,0,2,1,4,1,Gd,9,Typ,1,Gd,BuiltIn,2007.0,Fin,3,668,TA,TA,Y,100,18,0,0,0,0,NaN,NaN,NaN,0,1,2008,WD,Normal,214000
436,437,50,RM,40.0,4400,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Norm,Norm,1Fam,1.5Fin,6,8,1920,1950,Gable,CompShg,Stucco,Stucco,None,0.0,TA,TA,BrkTil,Fa,TA,No,Unf,0,Unf,0,648,648,GasA,TA,Y,FuseA,734,384,0,1118,0,0,1,0,2,1,TA,6,Typ,0,NaN,Detchd,1990.0,Unf,2,440,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,10,2006,WD,Normal,116000
610,611,60,RL,NaN,11050,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,PosN,Norm,1Fam,2Story,9,5,2000,2000,Hip,CompShg,VinylSd,VinylSd,BrkFace,204.0,Gd,TA,PConc,Ex,TA,Mn,GLQ,904,Unf,0,536,1440,GasA,Ex,Y,SBrkr,1476,677,0,2153,1,0,2,1,3,1,Ex,8,Typ,2,Ex,Attchd,2000.0,Fin,3,736,TA,TA,Y,253,142,0,0,0,0,NaN,NaN,NaN,0,5,2009,WD,Normal,313000
914,915,160,FV,30.0,3000,Pave,Pave,Reg,Lvl,AllPub,Inside,Gtl,Somerst,Norm,Norm,TwnhsE,2Story,6,5,2009,2009,Gable,CompShg,VinylSd,VinylSd,Stone,76.0,Gd,TA,PConc,Gd,TA,Av,GLQ,294,Unf,0,318,612,GasA,Ex,Y,SBrkr,612,612,0,1224,0,0,2,1,2,1,Gd,4,Typ,0,NaN,Detchd,2009.0,RFn,2,528,TA,TA,Y,0,234,0,0,0,0,NaN,NaN,NaN,0,6,2009,New,Partial,173733
1265,1266,160,FV,35.0,3735,Pave,NaN,Reg,Lvl,AllPub,FR3,Gtl,Somerst,Norm,Norm,TwnhsE,2Story,7,5,1999,1999,Hip,CompShg,MetalSd,MetalSd,BrkFace,218.0,Gd,TA,PConc,Gd,TA,No,GLQ,450,Unf,0,241,691,GasA,Ex,Y,SBrkr,713,739,0,1452,1,0,2,1,3,1,Gd,6,Typ,0,NaN,Detchd,1999.0,Unf,2,506,TA,TA,Y,0,34,0,0,0,0,NaN,NaN,NaN,0,3,2006,WD,Normal,183900
1418,1419,20,RL,71.0,9204,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,5,1963,1963,Gable,CompShg,HdBoard,HdBoard,None,0.0,TA,TA,CBlock,TA,TA,No,BLQ,25,Rec,872,247,1144,GasA,TA,Y,SBrkr,1144,0,0,1144,1,0,1,1,3,1,TA,6,Typ,0,NaN,Detchd,1962.0,Unf,1,336,TA,TA,Y,0,88,0,0,0,0,NaN,NaN,NaN,0,8,2008,COD,Normal,124000
822,823,60,RL,NaN,12394,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Gilbert,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Gd,TA,Gd,Unf,0,Unf,0,847,847,GasA,Ex,Y,SBrkr,847,886,0,1733,0,0,2,1,3,1,Gd,7,Typ,1,Gd,BuiltIn,2003.0,Fin,2,433,TA,TA,Y,100,48,0,0,0,0,NaN,NaN,NaN,0,10,2007,WD,Family,225000


In [0]:
df.drop(columns=['Id'], inplace=True)

In [0]:
# drop null-value columns
df_strip = df.drop(columns=df.columns[df.isna().any()].tolist())

In [112]:
df_strip.shape

(1460, 61)

In [0]:
#df_strip.dtypes

In [113]:
# do One hot encoding on objects
objects = list(df_strip.select_dtypes(exclude=["number"]))
df_strip = df_strip.join(pd.get_dummies(df_strip[objects]))
# drop object columns
df_strip = df_strip.drop(objects, axis=1)
df_strip.shape

(1460, 216)

In [0]:
# do correlation & keep positively correlated columns
corr_matrix = df_strip.corr().sort_values('SalePrice', ascending=False)
df_corr = pd.DataFrame(corr_matrix.SalePrice[:-1])
df_corr = df_corr[df_corr.SalePrice > 0.0].T
X_list = list(df_corr)

In [115]:
df_strip['FlattenedSalePrice'] = np.log(df_strip['SalePrice'])
df_strip.shape

(1460, 217)

In [116]:
## Log-Linear Regression
# Separate dependent and independent variables
y = df_strip.loc[:, ['FlattenedSalePrice']].values
X = df_strip.loc[:, X_list].values

# Slit into test and train datasets
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=.5, random_state=42)

# fit model using train datasets
model = LinearRegression()
model.fit(X_train, Y_train)

# Create new predictions using x_test
y_pred = model.predict(X_test)

# Measure Accuracy using y_test and y_pred
RMSE = (np.sqrt(mean_squared_error(Y_test, y_pred)))
R2 = r2_score(Y_test, y_pred)

print('RMSE is {}'.format(RMSE))
print('R^2 is {}'.format(R2))

RMSE is 0.09641059577569994
R^2 is 0.9442598828942611


In [0]:
X_list